In [1]:
import pulp

In [2]:
import pandas as pd
import numpy as np

## Data processing

In [3]:
#raw data
choices = [['A', 1,2,3], ['B', 3,2,1]]

In [4]:
df_preferences = pd.DataFrame.from_records(choices, 
                          columns=['Person']+["Option_{0}".format(i) for i in range(1,len(choices[1]))])
df_preferences = df_preferences.set_index('Person')

In [5]:
mat_pref = df_preferences.values.tolist()

In [34]:
#jiggle to randomly ensure no joint preferences
mat_pref = mat_pref + np.random.random(np.shape(mat_pref))/100

array([[1.00141761, 2.00702954, 3.00283049],
       [3.00454745, 2.00948842, 1.00301565]])

## Instantiate Model

In [6]:
model = pulp.LpProblem("Selection maxi-minimising cost", pulp.LpMaximize)

## Set Objective

In [7]:
aux_variable = pulp.LpVariable('Obj', cat = 'Continuous')
model += aux_variable, 'Maximin'

In [8]:
model

Selection maxi-minimising cost:
MAXIMIZE
1*Obj + 0
VARIABLES
Obj free Continuous

# Set free variable

In [9]:
peeps = range(len(df_preferences.index))
options = range(len(df_preferences.columns))

In [10]:
# allocation matrix
allocation = pulp.LpVariable.matrix('allocation', (peeps, options), cat='Binary')

In [11]:
allocation

[[allocation_0_0, allocation_0_1, allocation_0_2],
 [allocation_1_0, allocation_1_1, allocation_1_2]]

In [12]:
allocation[0][1]

allocation_0_1

## Add constraints

In [13]:
# Aux is minimum of indivdual costs
for i in peeps:
    model += aux_variable <= -pulp.lpDot(allocation[i], mat_pref[i])

In [14]:
# Only one project
for i in peeps:
    model += pulp.lpSum(allocation[i]) == 1

In [15]:
group_max = 2
group_min = 0
for j in options:
    model += pulp.lpSum(allocation[i][j] for i in peeps) <= group_max
    model += pulp.lpSum(allocation[i][j] for i in peeps) >= group_min

In [16]:
model

Selection maxi-minimising cost:
MAXIMIZE
1*Obj + 0
SUBJECT TO
_C1: Obj + allocation_0_0 + 2 allocation_0_1 + 3 allocation_0_2 <= 0

_C2: Obj + 3 allocation_1_0 + 2 allocation_1_1 + allocation_1_2 <= 0

_C3: allocation_0_0 + allocation_0_1 + allocation_0_2 = 1

_C4: allocation_1_0 + allocation_1_1 + allocation_1_2 = 1

_C5: allocation_0_0 + allocation_1_0 <= 2

_C6: allocation_0_0 + allocation_1_0 >= 0

_C7: allocation_0_1 + allocation_1_1 <= 2

_C8: allocation_0_1 + allocation_1_1 >= 0

_C9: allocation_0_2 + allocation_1_2 <= 2

_C10: allocation_0_2 + allocation_1_2 >= 0

VARIABLES
Obj free Continuous
0 <= allocation_0_0 <= 1 Integer
0 <= allocation_0_1 <= 1 Integer
0 <= allocation_0_2 <= 1 Integer
0 <= allocation_1_0 <= 1 Integer
0 <= allocation_1_1 <= 1 Integer
0 <= allocation_1_2 <= 1 Integer

In [42]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [52]:
pd.DataFrame.from_records([[allocation[i][j].varValue for j in options] for i in peeps],
                         columns=["Option_{0}".format(i) for i in range(1,len(choices[1]))])

,Option_1,Option_2,Option_3
0,1.0,0.0,0.0
1,0.0,0.0,1.0
